In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from datetime import datetime
from pyspark.sql.functions import broadcast
# startTime = datetime.now()

In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy 
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import confusion_matrix
import matplotlib.mlab as mlab
%matplotlib inline




Bad key "backend.qt4" on line 43 in
/etc/matplotlib/matplotlibrc.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.2.2/matplotlibrc.template
or from the matplotlib source distribution


In [4]:
import pandas as pd
from pandas import DataFrame
import numpy as np 
import scipy 
import scipy.stats as ss
from scipy.stats import chi2_contingency
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

In [5]:
conf = SparkConf()

conf = conf.set("spark.executor.instances", '8')
conf = conf.set("spark.executor.cores", '4')
conf = conf.set("spark.executor.memory", "60G")
# conf = conf.set("spark.jars", "db2jcc4.jar")
conf = conf.set("spark.sql.legacy.allowCreatingManagedTableUsingNonemptyLocation","true")
conf = conf.set("spark.sql.broadcastTimeout",  "900")
conf = conf.set("spark.driver.maxResultSize", "8g")

spark = SparkSession.builder \
    .config(conf=conf) \
    .config('spark.dynamicAllocation.enabled', 'false') \
    .config('spark.kryoserializer.buffer.max', '2047') \
    .config('spark.io.compression.codec', 'snappy') \
    .config('spark.blacklist.enabled', 'false') \
    .config('spark.dynamicAllocation.enabled', 'false') \
    .master("yarn") \
    .appName("---") \
    .config('spark.yarn.executor.memoryOverhead', '10g') \
    .enableHiveSupport() \
    .getOrCreate()

# INPUT DATA

In [73]:
# INPUT DATA
df_te_chen_seg_rekom=spark.sql("""


SELECT 
    status_customer,
    count(status_customer) as count_cus,
    rekomendasi,
    channel,
    segmentasi,
    status_kyc,
    jenis_kelamin,
    sum(jumlah_transaksi) as jumlah_transaksi,
    sum(total_amount) as total_amount,
    sum(total_berat) as total_berat,
    tgl_buka,
    year(tgl_buka) as tahun,
    case when (total_amount <10000) then
              '<Rp 10.000'
         when (total_amount <= 50000) then
              'Rp 10.000 - Rp 50.000' 
         when (total_amount <=100000) then
              'Rp 50.001 - Rp 100.000' 
         when (total_amount > 100000) then
              '> Rp 100.000'
    end as range 
    
FROM dev_staging.dm_profilling_te_channel_detail
GROUP BY status_customer, rekomendasi, channel, segmentasi,
    status_kyc,
    jenis_kelamin,
    total_amount,
    jumlah_transaksi,
    total_berat,
    tgl_buka

""")

type(df_te_chen_seg_rekom)
#df.show()

pyspark.sql.dataframe.DataFrame

In [74]:
df_te_chen_seg_rekom.show()

+------------------+---------+---------------+--------------------+----------+----------+-------------+----------------+------------------+--------------------+-------------------+-----+--------------------+
|   status_customer|count_cus|    rekomendasi|             channel|segmentasi|status_kyc|jenis_kelamin|jumlah_transaksi|      total_amount|         total_berat|           tgl_buka|tahun|               range|
+------------------+---------+---------------+--------------------+----------+----------+-------------+----------------+------------------+--------------------+-------------------+-----+--------------------+
|CUSTOMER_BELUM_KYC|      196|      Not_Churn|              SHOPEE| Uncertain|         0|    Perempuan|             196|107133.60000000002|              0.1176|2020-12-24 00:00:00| 2020|          <Rp 10.000|
|CUSTOMER_BELUM_KYC|        1|      Not_Churn|              SHOPEE|  Frequent|         0|    Perempuan|               8|            9334.6|0.029700000000000004|2020-10-

In [75]:
df_te_chen_seg_rekom.count()

597786

In [65]:
# temp table
df_te_chen_seg_rekom.registerTempTable('table1')

In [67]:
spark.sql(
"""
select segmentasi, count(segmentasi) from table1
group by segmentasi
"""
).show(10, False)

+------------+-----------------+
|segmentasi  |count(segmentasi)|
+------------+-----------------+
|Valuable    |824              |
|Spenders    |503              |
|null        |0                |
|Shopper     |151469           |
|Frequent    |46094            |
|AboutToSleep|1934             |
|FirstTime   |211766           |
|Uncertain   |217487           |
|Best        |13817            |
+------------+-----------------+



In [42]:
df_te_chen_seg_rekom.count()

2372408

# CEK DATA NULL

In [76]:
import databricks.koalas as ks
kdf= df_te_chen_seg_rekom.to_koalas()

In [77]:
kdf.info()

databricks.koalas.frame.DataFrame
Index: 597786 entries, 0 to 597785
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   status_customer   597786 non-null  object        
 1   count_cus         597786 non-null  int64         
 2   rekomendasi       597786 non-null  object        
 3   channel           394576 non-null  object        
 4   segmentasi        459361 non-null  object        
 5   status_kyc        489722 non-null  object        
 6   jenis_kelamin     494596 non-null  object        
 7   jumlah_transaksi  597786 non-null  int64         
 8   total_amount      498620 non-null  float64       
 9   total_berat       498620 non-null  float64       
 10  tgl_buka          597782 non-null  datetime64[ns]
 11  tahun             597782 non-null  int32         
 12  range             498620 non-null  object        
dtypes: datetime64[ns](1), float64(2), int32(1), int64(2), object(7)

In [78]:
kdf.describe()

,count_cus,jumlah_transaksi,total_amount,total_berat,tahun
count,597786.000000,597786.00000,4.986200e+05,498620.000000,597782.000000
mean,3.968792,63.01164,1.469679e+06,2.266612,2020.183293
std,33.974082,1227.04095,5.723643e+07,77.076402,0.769210
min,1.000000,1.00000,0.000000e+00,0.000100,2018.000000
25%,1.000000,2.00000,1.235730e+04,0.017200,2020.000000
50%,1.000000,6.00000,5.034060e+04,0.072900,2020.000000
75%,1.000000,32.00000,2.852000e+05,0.399800,2021.000000
max,2934.000000,321239.00000,3.857528e+10,52605.027400,2021.000000


In [79]:
isna_rep = kdf.isna().sum().to_frame().reset_index()
isna_rep.columns = ['var','sum_na']
isna_rep['prs']=isna_rep['sum_na']/len(kdf)*100
isna_rep

,var,sum_na,prs
0,status_customer,0,0.000000
1,count_cus,0,0.000000
2,rekomendasi,0,0.000000
3,channel,203210,33.993770
4,segmentasi,138425,23.156280
5,status_kyc,108064,18.077372
6,jenis_kelamin,103190,17.262030
7,jumlah_transaksi,0,0.000000
8,total_amount,99166,16.588880
9,total_berat,99166,16.588880


In [43]:
kdf.rekomendasi.value_counts() 

Not_Churn                 2289462
Low Cost Maintenance        65487
Prepare to Lose             16080
Sustained Relationship       1269
Aggressively Defend           110
Name: rekomendasi, dtype: int64

# SAVE TABEL

In [64]:
# saving

df_te_chen_seg_rekom.write.format('parquet').mode('overwrite').saveAsTable('dev_datamart.dm_profilling_te_channel_segmentasi_rekomendasi')

# CEK TABEL

In [65]:
# INPUT DATA
coba=spark.sql("""


SELECT 
status_kyc,
count(status_kyc)

FROM dev_datamart.dm_profilling_te_channel_segmentasi_rekomendasi
group by status_kyc


""")

In [66]:
coba.show()

+----------+-----------------+
|status_kyc|count(status_kyc)|
+----------+-----------------+
|         0|           301578|
|      null|                0|
|         1|           188144|
+----------+-----------------+



In [23]:
import databricks.koalas as ks
kcoba= coba.to_koalas()

In [24]:
isna_rep_coba = kcoba.isna().sum().to_frame().reset_index()
isna_rep_coba.columns = ['var','sum_na']
isna_rep_coba['prs']=isna_rep_coba['sum_na']/len(kcoba)*100
isna_rep_coba

,var,sum_na,prs
0,status_customer,0,0.000000
1,count_rokemndasi,0,0.000000
2,rekomendasi,0,0.000000
3,channel,1789650,75.436013
4,segmentasi,1728514,72.859053
5,norek,0,0.000000
6,status_kyc,1674680,70.589882
7,jenis_kelamin,1669573,70.374615
8,jumlah_transaksi,0,0.000000
9,total_amount,1665957,70.222196


In [25]:
kcoba.info()

databricks.koalas.frame.DataFrame
Index: 2372408 entries, 0 to 2372407
Data columns (total 14 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   status_customer   2372408 non-null  object        
 1   count_rokemndasi  2372408 non-null  int64         
 2   rekomendasi       2372408 non-null  object        
 3   channel           582758 non-null   object        
 4   segmentasi        643894 non-null   object        
 5   norek             2372408 non-null  object        
 6   status_kyc        697728 non-null   object        
 7   jenis_kelamin     702835 non-null   object        
 8   jumlah_transaksi  2372408 non-null  int64         
 9   total_amount      706451 non-null   float64       
 10  total_berat       706451 non-null   float64       
 11  tgl_buka          2371544 non-null  datetime64[ns]
 12  Tahun             2371544 non-null  int32         
 13  range             706451 non-null   object        


In [26]:
kcoba.shape

(2372408, 14)

In [20]:
kcoba = kcoba.dropna()
kcoba.shape

(510055, 11)

In [21]:
kcoba.head(15)

,count_rokemndasi,rekomendasi,channel,segmentasi,norek,status_kyc,jenis_kelamin,jumlah_transaksi,total_amount,total_berat,periode
0,1,Not_Churn,SHOPEE,FirstTime,1285021620004945,0,Perempuan,2,5127.8,0.0059,2021
1,1,Not_Churn,SHOPEE,Shopper,1282120620004054,0,Perempuan,6,279879.4,0.3182,2020
2,1,Not_Churn,SHOPEE,FirstTime,1303221620004709,0,Laki-laki,2,3073.0,0.0035,2021
3,1,Not_Churn,TOKOPEDIA,Uncertain,1232119653813929,0,Laki-laki,3,5403.6,0.0076,2019
4,1,Not_Churn,TOKOPEDIA,Frequent,1232119654912399,0,Perempuan,216,1131494.9,2.9526,2019
5,1,Not_Churn,SHOPEE,Shopper,1247321620007048,0,Perempuan,5,109308.3,0.1236,2021
6,1,Not_Churn,SHOPEE,Shopper,1410720620013321,0,Perempuan,5,18467.3,0.0609,2020
7,1,Prepare to Lose,SHOPEE,FirstTime,1288021620004967,1,Laki-laki,5,11117.7,0.0127,2021
8,1,Not_Churn,SHOPEE,Frequent,1112020620005042,1,Perempuan,6,5934.3,0.0189,2020
9,1,Low Cost Maintenance,SHOPEE,Uncertain,1132020620002409,0,Perempuan,4,53659.0,0.0921,2020


In [22]:
kcoba.periode.value_counts() 

2020    233948
2021    217751
2019     58356
Name: periode, dtype: int64

In [23]:
kcoba.segmentasi.value_counts() 

FirstTime       181871
Uncertain       172413
Shopper         110457
Frequent         36378
Best              6743
AboutToSleep      1234
Valuable           554
Spenders           405
Name: segmentasi, dtype: int64

In [7]:
coba2=spark.sql("""


SELECT 
    tgl_buka,
    year(tgl_buka)

FROM dev_staging.dm_profilling_te_channel_detail


""")

In [8]:
coba2.show()

+-------------------+----------------------------+
|           tgl_buka|year(CAST(tgl_buka AS DATE))|
+-------------------+----------------------------+
|2021-04-17 00:00:00|                        2021|
|2019-04-22 00:00:00|                        2019|
|2019-06-12 00:00:00|                        2019|
|2020-06-26 00:00:00|                        2020|
|2020-12-01 00:00:00|                        2020|
|2019-10-18 00:00:00|                        2019|
|2021-06-26 00:00:00|                        2021|
|2021-09-25 00:00:00|                        2021|
|2021-01-29 00:00:00|                        2021|
|2019-04-27 00:00:00|                        2019|
|2020-05-20 00:00:00|                        2020|
|2020-08-03 00:00:00|                        2020|
|2021-04-02 00:00:00|                        2021|
|2020-08-04 00:00:00|                        2020|
|2020-04-16 00:00:00|                        2020|
|2020-09-05 00:00:00|                        2020|
|2020-05-14 00:00:00|          

# cek tabel baru

In [6]:
coba=spark.sql("""


SELECT 
*
FROM dev_staging.dm_profilling_te_channel_detail
""")

In [7]:
coba.show()

+---------------+--------+---------+----------------+---------+----------+-------------------+-----------+----------------+----------+----------+----+-----+--------+----------+----------+-------------------+------------+------------------+---------+--------+-----+-------+-----------------+----------+----------+-------------+----------+-----------------+---------+----------+-----------+-----------+
|status_customer|     cif|create_by|           norek|CLIENT_ID|CHANNEL_ID|             tgl_OP|TARGET_CUST|jumlah_transaksi|status_kyc|source_kyc|nama|no_id| user_id|branch_kyc|status_rek|           tgl_buka|total_amount|       total_berat|  channel|username|agama|profesi|jumlah_tanggungan|pendidikan|range_gaji|jenis_kelamin|range_usia|dukcapil_verified|region_nm|segmentasi|rekomendasi|churn_proba|
+---------------+--------+---------+----------------+---------+----------+-------------------+-----------+----------------+----------+----------+----+-----+--------+----------+----------+-----------

In [8]:
coba.count()

2372488

In [9]:
import databricks.koalas as ks
kdf= coba.to_koalas()

In [10]:
kdf.info()

databricks.koalas.frame.DataFrame
Index: 2372488 entries, 0 to 2372487
Data columns (total 33 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   status_customer    2372488 non-null  object        
 1   cif                722622 non-null   object        
 2   create_by          722622 non-null   object        
 3   norek              2372488 non-null  object        
 4   CLIENT_ID          2372488 non-null  object        
 5   CHANNEL_ID         2372488 non-null  object        
 6   tgl_OP             2372488 non-null  datetime64[ns]
 7   TARGET_CUST        2372299 non-null  int32         
 8   jumlah_transaksi   2372488 non-null  int64         
 9   status_kyc         697808 non-null   object        
 10  source_kyc         220805 non-null   object        
 11  nama               702918 non-null   object        
 12  no_id              702917 non-null   object        
 13  user_id            115412 non-null  

In [11]:
kdf.describe()

,TARGET_CUST,jumlah_transaksi,total_amount,total_berat,jumlah_tanggungan,churn_proba
count,2.372299e+06,2.372488e+06,7.065300e+05,706530.000000,3315.000000,82986.000000
mean,4.050935e+05,1.587678e+01,1.037198e+06,1.599618,1.867300,0.650714
std,7.686303e+04,6.151557e+02,4.808672e+07,64.756872,1.357115,0.105135
min,5.000000e+03,1.000000e+00,0.000000e+00,0.000100,1.000000,0.500000
25%,2.800000e+05,1.000000e+00,5.027400e+03,0.005700,1.000000,0.584100
50%,4.500000e+05,2.000000e+00,1.658630e+04,0.025000,2.000000,0.618000
75%,4.500000e+05,7.000000e+00,1.251543e+05,0.172000,2.000000,0.709100
max,4.500000e+05,3.212390e+05,3.857528e+10,52605.027400,35.000000,0.966200


In [85]:
kdf.jumlah_norek.value_counts() 

1       753666
2        14479
3         6909
4         4314
5         3135
6         2261
7         1810
8         1465
9         1272
10        1023
11         906
12         768
13         708
14         626
15         551
16         478
17         474
19         401
18         373
20         353
21         308
22         293
23         280
24         280
25         258
26         237
27         217
28         212
29         194
30         193
34         176
31         154
33         153
36         153
32         151
38         141
37         137
35         136
40         112
41         110
39         104
42         101
47          94
50          93
45          91
44          89
48          88
43          86
46          86
49          85
56          79
51          78
52          78
54          76
55          74
59          66
60          61
57          60
53          60
58          59
70          56
67          55
68          54
63          53
66          53
61          52
62        

In [13]:
isna_rep = kdf.isna().sum().to_frame().reset_index()
isna_rep.columns = ['var','sum_na']
isna_rep['%']=isna_rep['sum_na']/len(kdf)*100
isna_rep

,var,sum_na,%
0,status_customer,0,0.000000
1,cif,1649866,69.541595
2,create_by,1649866,69.541595
3,norek,0,0.000000
4,CLIENT_ID,0,0.000000
5,CHANNEL_ID,0,0.000000
6,tgl_OP,0,0.000000
7,TARGET_CUST,189,0.007966
8,jumlah_transaksi,0,0.000000
9,status_kyc,1674680,70.587501


In [ ]:
dev_staging.dm_profilling_te_channel_detail

In [ ]:
# INPUT DATA
df_te_chen_seg_rekom=spark.sql("""


SELECT 
    status_customer,
    count(status_customer) as status_cus,
    rekomendasi,
    count(rekomendasi) as count_rokemndasi,
    channel,
    count(channel) as count_channel,
    segmentasi,
    count(segmentasi) as count_segmentasi,
    status_kyc,
    count(status_kyc) as count_status_kyc,
    jenis_kelamin,
    count(jenis_kelamin) as count_jenis_kelamin,
    sum(jumlah_transaksi) as jumlah_transaksi,
    sum(total_amount) as total_amount,
    sum(total_berat) as total_berat,
    tgl_buka,
    year(tgl_buka) as Tahun,
    case when (total_amount <'10000') then
              '<Rp 10.000'
         when (total_amount >= '10000' and total_amount <='50000') then
              'Rp 10.000 - Rp 50.000' 
         when (total_amount >= '50000' and total_amount <='100000') then
              'Rp 50.001 - Rp 100.000' 
         when (total_amount > '100000') then
              '> Rp 100.000'
    end as range 
    
FROM dev_staging.dm_profilling_te_channel_detail
GROUP BY status_customer, rekomendasi, channel, segmentasi,
    status_kyc,
    jenis_kelamin,
    total_amount,
    jumlah_transaksi,
    total_berat,
    tgl_buka

""")

type(df_te_chen_seg_rekom)
#df.show()